In [13]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor
import datetime as dt
import pickle

In [14]:
AHBA_exprs = pd.read_csv('./AHBA_exprs_noNorm.csv', index_col=0).T
#AHBA_exprs = pd.read_csv('./AHBA_exprs_SRS.csv', index_col=0).T

In [3]:
target_names = AHBA_exprs.columns
tf_names = [ tf.strip() for tf in open('./hs_hgnc_tfs.txt') ]
tf_names = target_names[target_names.isin(tf_names)]
tf_exprs = AHBA_exprs[tf_names]

In [ ]:
ET_KWARGS = {
    'n_jobs': 4,
    'n_estimators': 1000,
    'max_features': 'sqrt'
}

RF_KWARGS = {
    'n_jobs': 4,
    'n_estimators': 1000,
    'max_features': 'sqrt'
}
SGBM_KWARGS = {
    'learning_rate': 0.01,
    'n_estimators': 1000,  # can be arbitrarily large
    'max_features': 0.1,
    'subsample': 0.9
}

for k, t_name in enumerate(target_names):
    if t_name in tf_names:
        select_idx = tf_names.isin([t_name])
        train_x = tf_exprs.loc[:, ~select_idx]
    else:
        train_x = tf_exprs
    train_y = AHBA_exprs[t_name]

    #et = ExtraTreesRegressor(random_state=666, **ET_KWARGS)
    #et.fit(train_x, train_y)
    #links_df = pd.DataFrame({'TF': train_x.columns, 'importance': et.feature_importances_})

    rf = RandomForestRegressor(random_state=666, **RF_KWARGS)
    rf.fit(train_x, train_y)
    links_df = pd.DataFrame({'TF': train_x.columns, 'importance': rf.feature_importances_})

    #gbm = GradientBoostingRegressor(random_state=666, **SGBM_KWARGS)
    #gbm.fit(train_x, train_y)
    #links_df = pd.DataFrame({'TF': train_x.columns, 'importance': gbm.feature_importances_*gbm.n_estimators})


    links_df['target'] = t_name
    clean_links_df = links_df[links_df.importance > 0].sort_values(by='importance', ascending=False)
    clean_links_df = clean_links_df[['TF', 'target', 'importance']]
    clean_links_df.to_pickle(f'./2.grn/{t_name}_grn.pkl')
    if k % 10 == 0:
        print(dt.datetime.now(), clean_links_df.shape[0])
    #break

In [15]:
AHBA_exprs.shape

(4434, 15746)

## 2. Analysis GRN

In [8]:
import loompy as lp
import pandas as pd

In [9]:
lf = lp.connect("./2.grn/AHBA_exprs_noNorm_SCENIC.loom", mode='r+', validate=False)



In [10]:
pd.DataFrame(lf.ca.RegulonsAUC, index=lf.ca.CellID)

,AHRR(-),ALX3(-),ANXA11(+),AR(-),ARID3A(-),ARNTL(-),ARX(+),ARX(-),ASCL1(+),ASCL2(+),...,ZNF808(-),ZNF821(+),ZNF821(-),ZNF84(+),ZSCAN10(+),ZSCAN21(+),ZSCAN22(+),ZSCAN29(+),ZXDB(-),ZXDC(+)
0,0.139771,0.000000,0.105325,0.0,0.0,0.000000,0.085666,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,0.116900,0.002426,0.125092,0.0,0.0,0.000000,0.082005,0.0,0.0,0.047861,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,0.133672,0.000000,0.085095,0.0,0.0,0.000000,0.055913,0.0,0.0,0.040520,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002995,0.014386
3,0.053367,0.000000,0.040458,0.0,0.0,0.000000,0.027872,0.0,0.0,0.018495,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002935,0.025005
4,0.063024,0.004428,0.110869,0.0,0.0,0.004909,0.064358,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.019786,0.031993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4429,0.126557,0.015979,0.095520,0.0,0.0,0.000000,0.091688,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005809,0.006762
4430,0.150953,0.000000,0.105040,0.0,0.0,0.000000,0.083551,0.0,0.0,0.004094,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
4431,0.150699,0.000000,0.104478,0.0,0.0,0.000000,0.089381,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000545,0.000000
4432,0.143329,0.015209,0.079115,0.0,0.0,0.000000,0.087918,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000998,0.005763


array([   0,    1,    2, ..., 4431, 4432, 4433])